In [1]:
# Check if python is 3.8.10
import csv
import json
import napari
import numpy as np
import os
from skimage import io
import sys

print(sys.version)
%load_ext autoreload
%autoreload 2

3.8.10 (default, May 19 2021, 11:01:55) 
[Clang 10.0.0 ]


In [2]:
PATH_FILE = "../index.json"

paths_dict = json.load(open(PATH_FILE, "r"));

# Standardize the images

As of now, we have a lot of images in different dimensions... As we expect a standard (squared, pixel dimensions: 550 px), we need to crop and scale. This is done using `napari`, an interactive python tool.

First we define the paths of all images to crop.

In [3]:
img_dirs = [paths_dict["IMAGE_SOURCE_FILES"]["WIKI"],
           paths_dict["IMAGE_SOURCE_FILES"]["NHMC"]]

cropped_dir = paths_dict["IMAGE_CROPPED_FILES"]["CROPPED_IMGS"]
cropping_logfile = paths_dict["IMAGE_CROPPED_FILES"]["CROPPING_INDEX"]

In [4]:
images = []
for img_dir in img_dirs:
    files = [os.path.join(img_dir, fl) for fl in os.listdir(img_dir) if fl.lower().endswith("jpg")]
    images.extend(files)

In [5]:
images

['../data/images_source/ESEB gia web game_WIKI_photos/Cariama cristata.jpg',
 '../data/images_source/ESEB gia web game_WIKI_photos/Leptosomus discolor.jpg',
 '../data/images_source/ESEB gia web game_WIKI_photos/Fulmarus glacialis.jpg',
 '../data/images_source/ESEB gia web game_WIKI_photos/Apaloderma vittatum.jpg',
 '../data/images_source/ESEB gia web game_WIKI_photos/Buceros rhinoceros.jpg',
 '../data/images_source/ESEB gia web game_WIKI_photos/Mesitornis unicolor.jpg',
 '../data/images_source/ESEB gia web game_WIKI_photos/Acanthisitta chloris.jpg',
 '../data/images_source/ESEB gia web game_WIKI_photos/Eurypyga helias.jpg',
 '../data/images_source/ESEB gia web game_WIKI_photos/Chelonia mydas.jpg',
 '../data/images_source/ESEB gia web game_WIKI_photos/Melopsittacus undulatus.jpg',
 '../data/images_source/ESEB gia web game_WIKI_photos/Phoenicopterus ruber.jpg',
 '../data/images_source/ESEB gia web game_WIKI_photos/Geospiza fortis.jpg',
 '../data/images_source/ESEB gia web game_WIKI_photo

Then we open the `napari` GUI and add all images. Therefore, we use comfortable functions that we define here and in `napari_utils.py`.

In [6]:
from napari_utils import NapariIMG
from napari_utils import rescale_to_square
from napari_utils import save_cropping_areas

In [7]:
import ipywidgets as widgets

def make_button():
    '''Install a simple button widget.
    '''
    layout = widgets.Layout(width='auto', height='40px')
    button = widgets.Button(
        value=False,
        description='Wait for prompt...',
        disabled=False,
        button_style='info',
        display='flex',
        flex_flow='column',
        align_items='stretch',
        layout=layout
    )
    return button

In [8]:
import asyncio

def wait_for_change(widget):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.get_state())
        widget.on_click(getvalue)
    widget.on_click(getvalue)
    return future

In [9]:
async def crop_all_images(button, image_list, output_dir, 
                          cropping_dict={}, force=False, skip_first_n=0,
                          cropping_csv_file_name=None):
    img_count = len(image_list)
    for i, img in enumerate(image_list):
        if skip_first_n > i : continue
        # open a napari viewer and load image
        viewer = napari.Viewer()
        nimg = NapariIMG(img, viewer, output_dir=output_dir)

        # add the square and manually adjust it
        nimg.add_square()

        # prompt a button to press when editing is done
        
        # wait for repsonse of editor
        button.description = f"Choose cropping selection for image ({i+1}/{img_count})"
        await wait_for_change(button)

        # compute the optimal square
        opti_square = nimg.optimize_square()
        
        # ... and show it ...
        nimg.plot_optimal_square()
        
        # until the editor decides to proceed
        if i == img_count : button.description = f"Crop last image"
        else : button.description = f"Crop and show next image ({i+2}/{img_count})"
        await wait_for_change(button)
        
        # store the cropping area and save cropped image, we resacale to 550x550
        cropping_dict[nimg.alias] = nimg.crop_dict
        nimg.save_cropped(rescaling_function=rescale_to_square, force=force,
                         cropping_csv_file_name=cropping_csv_file_name)
        
        # close the viewer to proceed
        viewer.close()

Now it is time to interactively decide how to crop the images.

In [10]:
if not os.path.exists(cropped_dir) : os.makedirs(cropped_dir)

button = make_button()
cropping_thread = asyncio.create_task(
    crop_all_images(button, images, cropped_dir, cropping_csv_file_name=cropping_logfile))
button

Button(button_style='info', description='Wait for prompt...', layout=Layout(height='40px', width='auto'), styl…

From the cropping thread we read out the dictionary that stores all cropping areas.

In [ ]:
cropping_dict = cropping_thread.result()
save_cropping_areas(cropping_dict, file_name, force=False, append=False)

So now we are done :)